
MINE-4101: Ciencia de Datos Aplicada

Universidad de los Andes

Nombre: Juan Carlos Tovar
Código: 202216714;
José Jorge Geles;
Edgar Javier Toquica G.

Fecha: Octubre 17, 2025



In [ ]:
from scipy import stats
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [ ]:
used_cars_df = pd.read_excel("KMC-70k-93-2024-Obes 19-conVel-DATA-SPSS-20250322.xlsx")

In [ ]:
used_cars_df.sample(5)

,Idenfinal,Iden_Codigo,Iden_Sede,periodosanalisis,Sistemadeaseguramiento,Iden_embarazoMultiple,Iden_FechaParto,CSP_SituaPareja,CSP_TipoVivienda,CSP_EscolaridadMadre,...,formulaversusLME3m,mixtaversusLME3m,underweight12m,BMInacermas2DE,Nutpadre,velocidadzscorepeso40_3m,velocidadzscore3m_40semOMS,velocidad12_9mesesOMS,velocidad9_6mesesOMS,velocidad6_3mesesOMS
64387,4012144,12144,4,2,2,0,#NULL!,2,#NULL!,3,...,0,0,0,0,1,1.824529,1.176982,-0.176679,#NULL!,#NULL!
1902,1001904,1904,1,3,2,0,#NULL!,3,#NULL!,1,...,#NULL!,#NULL!,#NULL!,0,1,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!
58558,4002482,2482,4,1,2,0,#NULL!,#NULL!,#NULL!,#NULL!,...,#NULL!,0,#NULL!,0,#NULL!,1.193638,0.331154,#NULL!,0.348524,0.552276
15327,1015456,15456,1,5,2,0,2016-04-30 00:00:00,3,2,3,...,#NULL!,0,#NULL!,0,0,0.94321,0.043981,#NULL!,-0.188428,0.835684
57792,4001523,1523,4,1,2,0,#NULL!,2,#NULL!,5,...,#NULL!,0,#NULL!,0,#NULL!,1.167598,0.456939,#NULL!,#NULL!,#NULL!


In [ ]:
used_cars_df.shape

(64801, 753)

In [ ]:
used_cars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64801 entries, 0 to 64800
Columns: 753 entries, Idenfinal to velocidad6_3mesesOMS
dtypes: int64(1), object(752)
memory usage: 372.3+ MB


In [ ]:
# Muestra un listado de las columnas que tienen valores nulos (NaN)
used_cars_df.isnull().sum()[used_cars_df.isnull().sum() > 0].sort_values(ascending=False)

,0
V313,64758
V356,64756
V248,64738
V360C,64720
V423,64718
V402C,64710
V381,64705
V411,64701
V338,64681
V296,64663


In [ ]:
# 1. Crear la serie de nulos a partir de used_cars_df
nulos_series = used_cars_df.isnull().sum()

# 2. Obtener el número total de filas del DataFrame original
total_filas = len(used_cars_df)

# 3. Filtrar solo las columnas con nulos y ordenarlas
nulos_df = nulos_series[nulos_series > 0].sort_values(ascending=False).reset_index()

# 4. Renombrar las columnas para mayor claridad
nulos_df.columns = ['Columna', 'Conteo_Nulos']

# 5. Agregar la nueva columna con el total de datos (Total_Datos)
nulos_df['Total_Datos'] = total_filas

# Opcional: Agregar el porcentaje de nulos para una mejor visualización
nulos_df['Porcentaje_Nulos'] = (nulos_df['Conteo_Nulos'] / nulos_df['Total_Datos']) * 100

# 6. Mostrar las 20 columnas con más nulos
print("--- 20 Columnas con Mayor Cantidad de Nulos ---")
print(nulos_df.head(20))

--- 20 Columnas con Mayor Cantidad de Nulos ---
   Columna  Conteo_Nulos  Total_Datos  Porcentaje_Nulos
0     V313         64758        64801         99.933643
1     V356         64756        64801         99.930557
2     V248         64738        64801         99.902779
3    V360C         64720        64801         99.875002
4     V423         64718        64801         99.871916
5    V402C         64710        64801         99.859570
6     V381         64705        64801         99.851854
7     V411         64701        64801         99.845681
8     V338         64681        64801         99.814818
9     V296         64663        64801         99.787040
10    V369         64645        64801         99.759263
11    V376         64641        64801         99.753090
12    V418         64615        64801         99.712967
13    V333         64589        64801         99.672845
14   V317C         64580        64801         99.658956
15    V326         64542        64801         99.600315


In [ ]:
# 1. Calcular el umbral de nulos (40% de las filas totales)
total_filas = len(used_cars_df)
umbral_conteo = 0.40 * total_filas

# 2. Identificar las columnas a borrar
# Se usa used_cars_df.isnull().sum() para obtener el conteo de nulos por columna
columnas_a_eliminar = used_cars_df.columns[used_cars_df.isnull().sum() > umbral_conteo]

# 3. Crear el nuevo DataFrame limpio
used_cars_df_limpio = used_cars_df.drop(columns=columnas_a_eliminar)

# 4. Mostrar el resumen de las operaciones
print("--- Resumen de Limpieza por Nulos (> 40%) ---")
print(f"Filas totales: {total_filas}")
print(f"Columnas originales: {used_cars_df.shape[1]}")
print(f"Umbral de nulos (40%): {int(umbral_conteo)} nulos")
print(f"Columnas eliminadas: {len(columnas_a_eliminar)}")
print(f"Columnas finales: {used_cars_df_limpio.shape[1]}")

--- Resumen de Limpieza por Nulos (> 40%) ---
Filas totales: 64801
Columnas originales: 753
Umbral de nulos (40%): 25920 nulos
Columnas eliminadas: 36
Columnas finales: 717


In [ ]:
print("\n--- Listado de Columnas ELIMINADAS (Superan el 40% de Nulos) ---")
print(columnas_a_eliminar.tolist())


--- Listado de Columnas ELIMINADAS (Superan el 40% de Nulos) ---
['V203', 'V227', 'V236E', 'V245', 'V248', 'V254', 'V275C', 'V280', 'V284', 'V288', 'V291', 'V296', 'V301A', 'V313', 'V317C', 'V322', 'V326', 'V330', 'V333', 'V338', 'V344A', 'V356', 'V360C', 'V365', 'V369', 'V373', 'V376', 'V381', 'V386A', 'V402C', 'V407', 'V411', 'V415', 'V418', 'V423', 'V430A']


In [ ]:
# 1. Definir el umbral de nulos (5% del total de filas)
total_filas = len(used_cars_df_limpio)
umbral_5_porciento = 0.05 * total_filas

# 2. Identificar las columnas a imputar
# Buscamos columnas que tengan un conteo de nulos entre 1 y el 5% del total de filas
nulos_conteo = used_cars_df_limpio.isnull().sum()
columnas_a_imputar = nulos_conteo[
    (nulos_conteo > 0) & (nulos_conteo < umbral_5_porciento)
].index.tolist()

if not columnas_a_imputar:
    print("✅ ¡No se encontraron columnas con nulos entre 0% y 5% en el DataFrame limpio. Ya estaban perfectas en ese rango!")
else:
    # 3. Separar las columnas identificadas por tipo de dato
    columnas_numericas = used_cars_df_limpio[columnas_a_imputar].select_dtypes(include=np.number).columns.tolist()
    columnas_categoricas = used_cars_df_limpio[columnas_a_imputar].select_dtypes(include=['object']).columns.tolist()

    # 4. Imputación de Columnas Numéricas con la MEDIANA
    # La mediana es ideal porque es resistente a los valores atípicos (outliers).
    for col in columnas_numericas:
        mediana_valor = used_cars_df_limpio[col].median()
        used_cars_df_limpio[col].fillna(mediana_valor, inplace=True)

    # 5. Imputación de Columnas Categóricas con la MODA
    # La moda (valor más frecuente) preserva la distribución de la categoría.
    for col in columnas_categoricas:
        # .mode()[0] obtiene el valor más frecuente
        moda_valor = used_cars_df_limpio[col].mode()[0]
        used_cars_df_limpio[col].fillna(moda_valor, inplace=True)

    print(f"--- Imputación Simple Completada ---")
    print(f"Total de columnas imputadas: {len(columnas_a_imputar)}")
    print(f"  - Numéricas (Mediana): {len(columnas_numericas)}")
    print(f"  - Categóricas (Moda): {len(columnas_categoricas)}")
    print("\n✅ ¡La imputación simple se realizó con éxito! Todos los nulos en este rango han sido eliminados.")

    # Verificación final
    nulos_verificacion = used_cars_df_limpio[columnas_a_imputar].isnull().sum().sum()
    print(f"Nulos restantes en estas columnas: {nulos_verificacion}")

✅ ¡No se encontraron columnas con nulos entre 0% y 5% en el DataFrame limpio. Ya estaban perfectas en ese rango!


In [ ]:
# 1. Obtener el número total de filas
total_filas = len(used_cars_df_limpio)

# 2. Definir los umbrales de conteo de nulos
umbral_bajo = 0.05 * total_filas  # 5%
umbral_alto = 0.40 * total_filas # 40%

# 3. Filtrar las columnas que caen en este rango
nulos_conteo = used_cars_df_limpio.isnull().sum()

columnas_pendientes = nulos_conteo[
    (nulos_conteo >= umbral_bajo) & (nulos_conteo <= umbral_alto)
].sort_values(ascending=False)

if columnas_pendientes.empty:
    print("✅ ¡Felicidades! No quedan columnas con nulos entre el 5% y el 40%. El DataFrame está limpio, solo quedan las variables perfectas.")
else:
    print("--- Columnas Pendientes de Imputación (5% - 40% de Nulos) ---")
    print("Conteo | Porcentaje de Nulos")
    print("---------------------------------------")
    for nombre, conteo in columnas_pendientes.items():
        porcentaje = (conteo / total_filas) * 100
        print(f"{nombre}: {conteo} nulos ({porcentaje:.2f}%)")

✅ ¡Felicidades! No quedan columnas con nulos entre el 5% y el 40%. El DataFrame está limpio, solo quedan las variables perfectas.


In [ ]:
# DataFrame limpio
df = used_cars_df_limpio

# 1. Identificar todas las columnas que son de tipo 'object'
columnas_object = df.select_dtypes(include=['object']).columns

# Contador para seguimiento
columnas_convertidas = 0
nulos_creados = 0

print(f"--- Iniciando Conversión Forzada en {len(columnas_object)} columnas 'object' ---")

# 2. Iterar sobre las columnas 'object'
for col in columnas_object:
    # 3. Intentar la conversión a numérico.
    # errors='coerce' convierte cualquier texto no numérico a NaN.
    # inplace=True no se usa con pd.to_numeric, asignamos de vuelta a la columna.
    col_convertida = pd.to_numeric(df[col], errors='coerce')

    # 4. Verificar si la conversión fue exitosa y si vale la pena el cambio:
    # Si la columna convertida tiene MENOS nulos que la original, o
    # si el número de valores no nulos es mayor al 95% del total de filas.

    # Condición de éxito: El número de valores NO nulos en la nueva columna es mayor que un umbral.
    # Esto evita convertir columnas que son realmente texto (ej. nombres).
    if col_convertida.notna().sum() > (0.95 * len(df)):

        # Conteo de nulos creados
        nulos_antiguos = df[col].isnull().sum()
        nulos_nuevos = col_convertida.isnull().sum()

        if nulos_nuevos > nulos_antiguos:
            nulos_creados += (nulos_nuevos - nulos_antiguos)

        # 5. Aplicar la conversión al DataFrame original
        df[col] = col_convertida
        columnas_convertidas += 1

print(f"--- Conversión Terminada ---")
print(f"Columnas convertidas de 'object' a numérico: {columnas_convertidas}")
print(f"Nuevos nulos creados por caracteres inválidos: {nulos_creados}")

# Vuelve a verificar: ¿la conversión creó nuevos nulos?
print("\n--- Verificación de los 5 principales nulos ---")
print(df.isnull().sum().sort_values(ascending=False).head())

--- Iniciando Conversión Forzada en 716 columnas 'object' ---
--- Conversión Terminada ---
Columnas convertidas de 'object' a numérico: 72
Nuevos nulos creados por caracteres inválidos: 44215

--- Verificación de los 5 principales nulos ---
anoxia5mn               2566
SGAprema                2367
CSP_EscolaridadMadre    2007
CSP_SituaPareja         1874
CP_SA_InfUrinaria       1641
dtype: int64


In [ ]:
# 1. Obtener el número total de filas
total_filas = len(used_cars_df_limpio)

# 2. Calcular el conteo de nulos por columna (TODAS las columnas)
nulos_conteo = used_cars_df_limpio.isnull().sum()

# 3. Filtrar solo las columnas que AÚN tengan nulos y ordenarlas
nulos_df_reporte = nulos_conteo[nulos_conteo > 0].sort_values(ascending=False).reset_index()

# 4. Asignar nombres a las columnas
nulos_df_reporte.columns = ['Variable', 'Conteo_Nulos']

# 5. Agregar las columnas de total y porcentaje
nulos_df_reporte['Total_Datos'] = total_filas
nulos_df_reporte['Porcentaje_Nulos'] = (nulos_df_reporte['Conteo_Nulos'] / nulos_df_reporte['Total_Datos']) * 100

print("--- REPORTE COMPLETO del Estado Actual de Nulos ---")
print("  (Ordenado de la variable con más nulos a la que menos tiene)")

if nulos_df_reporte.empty:
    print("✅ ¡El DataFrame está completamente limpio! No queda ninguna variable con nulos.")
else:
    # Mostrar todas las variables pendientes
    print(nulos_df_reporte)

--- REPORTE COMPLETO del Estado Actual de Nulos ---
  (Ordenado de la variable con más nulos a la que menos tiene)
                Variable  Conteo_Nulos  Total_Datos  Porcentaje_Nulos
0              anoxia5mn          2566        64801          3.959815
1               SGAprema          2367        64801          3.652721
2   CSP_EscolaridadMadre          2007        64801          3.097174
3        CSP_SituaPareja          1874        64801          2.891931
4        CP_SA_InfGineco          1641        64801          2.532368
..                   ...           ...          ...               ...
67             CONSULT12            16        64801          0.024691
68              filter_$            16        64801          0.024691
69           PrimaryLast            16        64801          0.024691
70               preterm            16        64801          0.024691
71                  LBWI            16        64801          0.024691

[72 rows x 4 columns]


In [ ]:
print("--- Iniciando Imputación Final de Nulos (< 5%) con BUENA PRÁCTICA ---")

# 1. Identificar todas las columnas que aún tienen nulos
columnas_con_nulos_final = used_cars_df_limpio.columns[used_cars_df_limpio.isnull().any()].tolist()

# 2. Separar por tipo de dato (Numéricas y Categóricas/Object)
nulos_numericos = used_cars_df_limpio[columnas_con_nulos_final].select_dtypes(include=np.number).columns.tolist()
nulos_categoricos = used_cars_df_limpio[columnas_con_nulos_final].select_dtypes(include=['object']).columns.tolist()

# 3. Imputación de Nulos Numéricos (MEDIANA)
if nulos_numericos:
    print(f"\nImputando {len(nulos_numericos)} Numéricas con la Mediana...")
    for col in nulos_numericos:
        mediana_valor = used_cars_df_limpio[col].median()

        # BUENA PRÁCTICA: Asignar el resultado de fillna directamente a la columna
        used_cars_df_limpio[col] = used_cars_df_limpio[col].fillna(mediana_valor)

# 4. Imputación de Nulos Categóricos (MODA)
if nulos_categoricos:
    print(f"Imputando {len(nulos_categoricos)} Categóricas con la Moda...")
    for col in nulos_categoricos:
        moda_valor = used_cars_df_limpio[col].mode()[0]

        # BUENA PRÁCTICA: Asignar el resultado de fillna directamente a la columna
        used_cars_df_limpio[col] = used_cars_df_limpio[col].fillna(moda_valor)

# 5. Verificación final de nulos
nulos_totales_restantes = used_cars_df_limpio.isnull().sum().sum()

print("\n--- ¡Fase de Imputación Completada! ---")
print(f"Nulos totales restantes en todo el DataFrame: {nulos_totales_restantes}")
print("Advertencias de asignación eliminadas. ✅")

--- Iniciando Imputación Final de Nulos (< 5%) con BUENA PRÁCTICA ---

Imputando 72 Numéricas con la Mediana...

--- ¡Fase de Imputación Completada! ---
Nulos totales restantes en todo el DataFrame: 0
Advertencias de asignación eliminadas. ✅


In [ ]:
# 1. Identificar todas las columnas que son numéricas (int o float)
df_num = used_cars_df_limpio.select_dtypes(include=np.number)
columnas_numericas = df_num.columns

outlier_reporte = {}
total_outliers = 0

print("--- Detección de Outliers (Método IQR) ---")

# 2. Iterar sobre cada columna numérica
for col in columnas_numericas:
    # 3. Calcular los cuartiles Q1 y Q3
    Q1 = df_num[col].quantile(0.25)
    Q3 = df_num[col].quantile(0.75)
    IQR = Q3 - Q1

    # 4. Definir los límites del outlier (1.5 * IQR)
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # 5. Contar los valores que están fuera de los límites
    outliers_conteo = df_num[
        (df_num[col] < limite_inferior) | (df_num[col] > limite_superior)
    ].shape[0]

    if outliers_conteo > 0:
        outlier_reporte[col] = outliers_conteo
        total_outliers += outliers_conteo

# 6. Crear un DataFrame con el reporte y ordenar por conteo
outliers_df = pd.Series(outlier_reporte).sort_values(ascending=False).reset_index()
outliers_df.columns = ['Variable', 'Conteo_Outliers']

# 7. Agregar porcentaje
outliers_df['Porcentaje_Outliers'] = (outliers_df['Conteo_Outliers'] / len(used_cars_df_limpio)) * 100

print(f"\nTotal de columnas numéricas revisadas: {len(columnas_numericas)}")
print(f"Columnas con Outliers: {len(outliers_df)}")
print(f"Conteo Total de Outliers: {total_outliers}")

print("\n--- Top 10 Columnas con Mayor Cantidad de Outliers ---")
print(outliers_df.head(10))

--- Detección de Outliers (Método IQR) ---

Total de columnas numéricas revisadas: 73
Columnas con Outliers: 53
Conteo Total de Outliers: 289171

--- Top 10 Columnas con Mayor Cantidad de Outliers ---
                 Variable  Conteo_Outliers  Porcentaje_Outliers
0  Sistemadeaseguramiento            23215            35.825064
1              edadmatcat            18019            27.806670
2           primariamadre            15513            23.939445
3                     BPN            15395            23.757349
4       oxigenoalaentrada            14703            22.689465
5                 toxemia            13667            21.090724
6               RCIUtalla            12694            19.589204
7               hospenURN            12026            18.558356
8      HD_DiasVenMecanica            11848            18.283668
9         CP_SA_InfGineco            11847            18.282125


In [ ]:
# Lista de variables que tienen outliers altos y son probablemente códigos/categorías:
columnas_a_categorizar = [
    'Sistemadeaseguramiento', 'edadmatcat', 'primariamadre', 'BPN',
    'oxigenoalaentrada', 'toxemia', 'RCIUtalla', 'hospenURN', 'CP_SA_InfGineco'
    # Agrega aquí cualquier otra columna que sepas que es un código o booleana (0/1)
]

print("--- Iniciando Reclasificación de Códigos a Tipo 'category' ---")
for col in columnas_a_categorizar:
    used_cars_df_limpio[col] = used_cars_df_limpio[col].astype('category')
print(f"✅ {len(columnas_a_categorizar)} columnas convertidas a tipo 'category'.")

# Ahora tu DataFrame tiene menos columnas numéricas que generan falsos positivos.

--- Iniciando Reclasificación de Códigos a Tipo 'category' ---
✅ 9 columnas convertidas a tipo 'category'.


In [ ]:
# Seleccionar solo las columnas numéricas reales
columnas_num_reales = used_cars_df_limpio.select_dtypes(include=np.number).columns

print("\n--- Iniciando Capping (Tratamiento de Outliers) en Numéricas Reales ---")
outliers_tratados = 0

for col in columnas_num_reales:
    Q1 = used_cars_df_limpio[col].quantile(0.25)
    Q3 = used_cars_df_limpio[col].quantile(0.75)
    IQR = Q3 - Q1

    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Aplicar Capping: Reemplazar valores fuera de los límites
    num_cambios = used_cars_df_limpio[col][
        (used_cars_df_limpio[col] < limite_inferior) | (used_cars_df_limpio[col] > limite_superior)
    ].count()

    used_cars_df_limpio[col] = np.where(
        used_cars_df_limpio[col] < limite_inferior, limite_inferior, used_cars_df_limpio[col]
    )
    used_cars_df_limpio[col] = np.where(
        used_cars_df_limpio[col] > limite_superior, limite_superior, used_cars_df_limpio[col]
    )
    outliers_tratados += num_cambios

print(f"✅ Tratamiento de Outliers (Capping) completado.")
print(f"Total de valores outliers reemplazados/tratados: {outliers_tratados}")


--- Iniciando Capping (Tratamiento de Outliers) en Numéricas Reales ---
✅ Tratamiento de Outliers (Capping) completado.
Total de valores outliers reemplazados/tratados: 152092
